<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achat_miniCPMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ai-bot-pro/achatbot.git -b feat/vision_voice

In [2]:
%cd /content/achatbot

/content/achatbot


In [ ]:
!git submodule update --init --recursive

In [31]:
!git checkout . && git pull origin feat/vision_voice

Updated 2 paths from the index
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 11 (delta 8), reused 11 (delta 8), pack-reused 0 (from 0)
Unpacking objects: 100% (11/11), 1.24 KiB | 424.00 KiB/s, done.
From https://github.com/ai-bot-pro/achatbot
 * branch            feat/vision_voice -> FETCH_HEAD
   e0740c5..8366557  feat/vision_voice -> origin/feat/vision_voice
Updating e0740c5..8366557
Fast-forward
 src/core/llm/transformers/manual_vision_voice_minicpmo.py | 6 +++++-
 src/modules/speech/tts/minicpmo_tts.py                    | 4 +++-
 2 files changed, 8 insertions(+), 2 deletions(-)


In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [6]:
!pip install -q dist/achatbot-0.0.8.10.11-py3-none-any.whl[llm_transformers_manual_vision_voice_minicpmo,grpc]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
  

# Prepare code and install AutoGPTQ


if use openbmb/MiniCPM-o-2_6-int4, need use AutoGPTQ lib

In [ ]:
%cd /content

/content


In [ ]:
!git clone https://github.com/OpenBMB/AutoGPTQ.git -b minicpmo
!cd AutoGPTQ && git checkout minicpmo && pip install -vvv --quiet --no-build-isolation -e .

# download ckpt


In [7]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [8]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential


Token is valid (permission: write).
The token `write` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `write`


In [9]:
!huggingface-cli download openbmb/MiniCPM-o-2_6 --quiet --local-dir /content/models/openbmb/MiniCPM-o-2_6


/content/models/openbmb/MiniCPM-o-2_6


In [ ]:
!huggingface-cli download openbmb/MiniCPM-o-2_6-int4 --quiet --local-dir /content/models/openbmb/MiniCPM-o-2_6-int4

/content/models/openbmb/MiniCPM-o-2_6-int4


# test

In [ ]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

# load omni model default, the default init_vision/init_audio/init_tts is True
# if load vision-only model, please set init_audio=False and init_tts=False
# if load audio-only model, please set init_vision=False
model = AutoModel.from_pretrained(
    '/content/models/openbmb/MiniCPM-o-2_6',
    trust_remote_code=True,
    attn_implementation='sdpa', # sdpa or flash_attention_2
    torch_dtype=torch.bfloat16,
    init_vision=False,
    init_audio=False,
    init_tts=True
)

model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained('/content/models/openbmb/MiniCPM-o-2_6', trust_remote_code=True)

model.init_tts()


In [54]:
!LLM_MODEL_NAME_OR_PATH=/content/models/openbmb/MiniCPM-o-2_6 \
    LLM_GEN_TEMPERATURE=0.0 \
    LLM_DEVICE=cuda LLM_TORCH_DTYPE=bfloat16 \
    TTS_TEXT="你叫什么名字?" \
    python -m unittest test.modules.speech.tts.test_minicpmo.TestMiniCPMoTTS.test_synthesize

2025-02-14 03:51:35,250 - chat-bot - INFO - /content/achatbot/src/common/factory.py:69 - get_engine_by_tag - use tts_minicpmo engine
2025-02-14 03:51:35,250 - chat-bot - INFO - /content/achatbot/src/common/factory.py:34 - get_instance - class: <class 'src.modules.speech.tts.minicpmo_tts.MiniCPMoTTS'> args: {'lm_model_name_or_path': '/content/models/openbmb/MiniCPM-o-2_6', 'lm_device_map': None, 'lm_device': 'cuda', 'lm_torch_dtype': 'bfloat16', 'lm_attn_impl': None, 'user_role': 'user', 'warnup_prompt': "Repeat the word 'weedge niu bi'.", 'warnup_steps': 1, 'init_chat_role': 'system', 'init_chat_prompt': '', 'lm_gen_seed': 42, 'lm_gen_max_new_tokens': 1024, 'lm_gen_min_new_tokens': 0, 'lm_gen_do_sample': False, 'lm_gen_temperature': 0.0, 'lm_gen_top_k': 1, 'lm_gen_top_p': 0.8, 'lm_gen_repetition_penalty': 1.0, 'lm_tokenizer_decode_batch_size': 60, 'chat_history_size': 10, 'lm_stream': True, 'model_type': 'chat_completion', 'lm_bnb_quant_type': 'int4', 'instruct_tpl': ''}
2025-02-14 03:

In [55]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_minicpmo.wav",autoplay=True)

# tts speaker with grpc


In [49]:
!python -m src.cmd.grpc.speaker.server.serve

2025-02-14 03:45:42,969 - chat-bot-tts-serve - INFO - /content/achatbot/src/cmd/grpc/speaker/server/serve.py:33 - serve - serve port: 50052 max_workers: 10
2025-02-14 03:45:43,005 - chat-bot-tts-serve - INFO - /content/achatbot/src/cmd/grpc/speaker/server/serve.py:45 - serve - Server started port: 50052
2025-02-14 03:46:14,734 - chat-bot-tts-serve - DEBUG - /content/achatbot/src/common/grpc/interceptors/authentication_server.py:43 - intercept_service - authorization, oligei-tts,_HandlerCallDetails(method='/chat_bot.tts.TTS/LoadModel', invocation_metadata=(_Metadatum(key='user-agent', value='grpc-python/1.70.0 grpc-c/45.0.0 (linux; chttp2)'), _Metadatum(key='request_streaming', value='u'), _Metadatum(key='response_streaming', value='u'), _Metadatum(key='authorization', value='oligei-tts')))
2025-02-14 03:46:14,735 - chat-bot-tts-serve - DEBUG - /content/achatbot/src/cmd/grpc/speaker/server/servicers/tts.py:41 - LoadModel - LoadModel request: tts_tag: "tts_minicpmo"
is_reload: true
json_

In [51]:
!TTS_TAG=tts_minicpmo \
    LLM_DEVICE=cuda LLM_GEN_TEMPERATURE=0.5 LLM_TORCH_DTYPE=bfloat16 \
    LLM_MODEL_NAME_OR_PATH=/content/models/openbmb/MiniCPM-o-2_6 \
    IS_SAVE=1 IS_RELOAD=1 python -m src.cmd.grpc.speaker.client

2025-02-14 03:47:37,149 - chat-bot-tts-client - DEBUG - /content/achatbot/src/cmd/grpc/speaker/client.py:59 - load_model - tts_tag: "tts_minicpmo"
is_reload: true
json_kwargs: "{\"lm_model_name_or_path\": \"/content/models/openbmb/MiniCPM-o-2_6\", \"lm_device_map\": null, \"lm_device\": \"cuda\", \"lm_torch_dtype\": \"bfloat16\", \"lm_attn_impl\": null, \"user_role\": \"user\", \"warnup_prompt\": \"Repeat the word \'weedge niu bi\'.\", \"warnup_steps\": 1, \"init_chat_role\": \"system\", \"init_chat_prompt\": \"\", \"lm_gen_seed\": 42, \"lm_gen_max_new_tokens\": 1024, \"lm_gen_min_new_tokens\": 0, \"lm_gen_do_sample\": false, \"lm_gen_temperature\": 0.5, \"lm_gen_top_k\": 1, \"lm_gen_top_p\": 0.8, \"lm_gen_repetition_penalty\": 1.0, \"lm_tokenizer_decode_batch_size\": 60, \"chat_history_size\": 10, \"lm_stream\": true, \"model_type\": \"chat_completion\", \"lm_bnb_quant_type\": \"int4\", \"instruct_tpl\": \"\"}"

2025-02-14 03:47:37,149 - chat-bot-tts-client - DEBUG - /content/achatbot

In [52]:
from IPython.display import Audio
Audio("/content/achatbot/records/grpc_tts_minicpmo.wav",autoplay=True)
